In [15]:
# import libraries
import numpy as np
import pandas as pd
import random
import thinkstats2
import thinkplot
import patsy
import re
import statsmodels.formula.api as smf

In [12]:
# Copy over defined functions from book, but altered them to use for my assignment

def GoMining(df):
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'prglngth ~ ' + name

            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError):
            continue

        variables.append((results.rsquared, name))

    return variables

def ReadVariables():
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

def MiningReport(variables, n=50):
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)

## Exercise 11-1

**Exercise:** Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [3]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

In [7]:
# Call the GoMining function which I altered to calculate R2 for prglngth variable
variables = GoMining(live)

In [8]:
# Use the functions to sort and display the list of the highest scoring R2 variables for prglngth
MiningReport(variables)

prglngth 1.0 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.8062434116139248 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
totalwgt_lb 0.12445743148120225
birthwgt_lb 0.11977307804917248 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.10372542204583246 LOW BIRTHWEIGHT - BABY 1
mosgest 0.09562431989592235 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i 0.022053775796472053 PRGLNGTH IMPUTATION FLAG
nbrnaliv 0.004577565785538473 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
anynurse 0.002452024883713211 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.002369183944671338 DURATION OF BREASTFEEDING IN WEEKS
pregend1 0.002249389433796045 BC-1 HOW PREGNANCY ENDED - 1ST MENTION
cmlastlb 0.0020431424422018285 CM FOR R'S MOST RECENT LIVE BIRTH
fmarcon5_i 0.001968159324254537 FMARCON5 IMPUTATION FLAG
evuseint 0.0018917527758633979 EG-1 USE ANY METHOD IN PREGNANCY INTERVAL?
gestasun_m 0.001657131955017932 BC-5 GESTATIONAL

In [ ]:
# Based on the list above: nbrnaliv - # of babies born alive had R2 of 0.005 & birthord - birth order had R2 of 0.001 are 
# the only variables I see that you would realistically know about a coworker. This doesn't quite match up with the 
# solution file, so I'm going to add in the respondent data and see what pops up then. I'm also going to increase the 
# list to top 50. 

In [9]:
import nsfg
live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

In [13]:
variables2 = GoMining(join)

In [14]:
MiningReport(variables2)

prglngth 1.0 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.8062434116139248 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
totalwgt_lb 0.12445743148120225
birthwgt_lb 0.11977307804917248 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.10372542204583246 LOW BIRTHWEIGHT - BABY 1
mosgest 0.09562431989592235 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i 0.022053775796472053 PRGLNGTH IMPUTATION FLAG
canhaver 0.006050495268195344 DF-1 PHYSICALLY DIFFICULT FOR R TO HAVE A BABY
datcon01_i 0.00581775529987516 DATCON01 IMPUTATION FLAG
con1mar1_i 0.005546376136237985 CON1MAR1 IMPUTATION FLAG
nbrnaliv 0.004577565785538473 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
mar1con1_i 0.0031508022538604408 MAR1CON1 IMPUTATION FLAG
anynurse 0.002452024883713211 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.002369183944671338 DURATION OF BREASTFEEDING IN WEEKS
pregend1 0.002249389433796045 BC-1 HOW PREGNANCY ENDED - 1ST MENT

In [ ]:
# Here we still have nbrnaliv - # of babies born alive had R2 of 0.005 & paydu - current living quarters owned or 
# rented had R2 of 0.002, but birthorder didn't make the list and a lot of this stuff is not public knowledge. 

In [20]:
# I'm going to run the stat summary on the second set of variables, since it was more comprehensive
results = smf.ols('prglngth ~ nbrnaliv>1 + paydu == 1', data=join).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     46.65
Date:                Thu, 29 Oct 2020   Prob (F-statistic):           6.98e-21
Time:                        19:05:45   Log-Likelihood:                -18252.
No. Observations:                8884   AIC:                         3.651e+04
Df Residuals:                    8881   BIC:                         3.653e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               38.8430      0.028   1375.690      0.000      38.788      38.898
nbrnaliv > 1[T.True]    -1.5093      0.164     -9.182      0.000      -1.832      -1.187
paydu == 1[T.True]       0.1175      0.040      2.933      0.003       0.039       0.196
==============================================================================
Omnibus:                     1556.606   Durbin-Watson:                   1.620
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6047.201
Skew:                          -0.834   Prob(JB):                         0.00
Kurtosis:                       6.681   Cond. No.                         9.36
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# The variables that I see to cause the most effect is Twins or not and what child this is for the mother.

## Exercise 11-3

**Exercise:** If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [41]:
# Create a formula for the regression test
formula = 'numbabes ~ ager + race + totincr + havedeg'
results = smf.poisson(formula, data=join).fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.590769
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 4239
Model:                        Poisson   Df Residuals:                     4234
Method:                           MLE   Df Model:                            4
Date:                Thu, 29 Oct 2020   Pseudo R-squ.:                 0.01333
Time:                        19:19:56   Log-Likelihood:                -6743.3
converged:                       True   LL-Null:                       -6834.4
Covariance Type:            nonrobust   LLR p-value:                 2.464e-38
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3978      0.075      5.338      0.000       0.252       0.544
ager           0.0206      0.002     11.993      0.000       0.017       0.024
race          -0.0433      0.019     -2.313      0.021      -0.080      -0.007
totincr       -0.0201      0.003     -6.999      0.000      -0.026      -0.014
havedeg        0.0126      0.005      2.382      0.017       0.002       0.023
==============================================================================
"""

In [39]:
# Create a pandas series to house the information we are going to use for the prediction
columns = ['ager', 'race','totincr', 'havedeg']
kids = pd.DataFrame([[35, 0, 15, 1]])
y = results.predict(kids)

PatsyError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error evaluating factor: NameError: name 'ager' is not defined
    numbabes ~ ager + C(race) + totincr + havedeg
               ^^^^

In [43]:
# I don't understand why age_r is not defined, it's in the file that lists the columns (AGER   %2f  "Age at interview")
# Oh duh, I forgot to name the columns in the pd series
columns = ['ager', 'race','totincr', 'havedeg']
kids = pd.DataFrame([[35, 1, 15, 1]], columns=columns)
y = results.predict(kids)
y

0    2.195517
dtype: float64

In [41]:
# So my prediction is that she has two kids.

## Exercise 11-4

**Exercise:** If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [46]:
# Create my formula for marital status
formula = 'rmarital ~ ager + race + havedip + totincr'
results = smf.mnlogit(formula, data=join).fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.100517
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8747
Model:                        MNLogit   Df Residuals:                     8722
Method:                           MLE   Df Model:                           20
Date:                Thu, 29 Oct 2020   Pseudo R-squ.:                  0.1541
Time:                        19:31:35   Log-Likelihood:                -9626.2
converged:                       True   LL-Null:                       -11380.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.2896      0.254      9.029      0.000       1.793       2.787
ager          -0.0606      0.006    -10.195      0.000      -0.072      -0.049
race          -0.4936      0.067     -7.375      0.000      -0.625      -0.362
havedip        0.1572      0.021      7.317      0.000       0.115       0.199
totincr       -0.1628      0.011    -14.443      0.000      -0.185      -0.141
------------------------------------------------------------------------------
rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -5.5371      0.841     -6.580      0.000      -7.186      -3.888
ager           0.1238      0.019      6.482      0.000       0.086       0.161
race          -0.0971      0.171     -0.570      0.569      -0.431       0.237
havedip        0.0302      0.056      0.540      0.589      -0.079       0.140
totincr       -0.3454      0.031    -11.127      0.000      -0.406      -0.285
------------------------------------------------------------------------------
rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.3215      0.289     -4.574      0.000      -1.888      -0.755
ager           0.0678      0.007     10.212      0.000       0.055       0.081
race          -0.2953      0.069     -4.255      0.000      -0.431      -0.159
havedip       -0.1725      0.027     -6.288      0.000      -0.226      -0.119
totincr       -0.2318      0.011    -20.875      0.000      -0.254      -0.210
------------------------------------------------------------------------------
rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9407      0.311      3.022      0.003       0.331       1.551
ager           0.0064      0.007      0.878      0.380      -0.008       0.021
race          -0.5191      0.079     -6.548      0.000      -0.674      -0.364
havedip        0.0284      0.026      1.082      0.279      -0.023       0.080
totincr       -0.3046      0.014    -21.565      0.000      -0.332      -0.277
------------------------------------------------------------------------------
rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.6543      0.242     27.483      0.000       6.180       7.129
ager          -0.0910      0.006    -15.823      0.000      -0.102      -0.080
race          -1.5358      0.066    -23.400      0.000      -1.664      -1.407
havedip        0.0282      0.021      1.349      0.177      -0.013       0.069
totincr       -0.3216      0.011    -28.109      0.000      -0.344      -0.299
==============================================================================
""

Make a prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

In [47]:
# Predict 
columns = ['ager', 'race', 'havedip', 'totincr']
married = pd.DataFrame([[25, 2, 1, 9]], columns=columns)
y = results.predict(married)
y

,0,1,2,3,4,5
0,0.630094,0.137846,0.002079,0.053013,0.044472,0.132496


In [ ]:
# I show a 63% chance she's married and a 14% chance she is cohabitating with a significant other. 
# My numbers may be different from the solution file because I used different variables such as havedip instead of educat.